# 3.2 LangChain에서 도구(tool) 활용 방법

LangChain에서 도구(tool)을 활용하는 방법을 알아봄  
이 강의에서는 도구를 활용하는 방법을 중점적으로 다룸  
- LangGraph에서의 도구 활용 방법은 LangChain의 문법을 따름

도구는 랭그래프에서 노드가 될 수 있음(ToolNode 활용)   
**랭체인에서 도구는 스키마(Name, Description, Expected arguments)를 가지는 함수를 뜻함**
- Description: docstring
- Expected arguments: typing

다시 말해 그냥 함수를 정의하면 도구로 쓸수 없다.   

그리고 @tool이라는 데코레이터를 사용해야한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4o',
)

small_llm = ChatOpenAI(
    model='gpt-4o-mini',
)

- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

tool 사용해보기
- invoke 함수를 사용하면 된다.

In [4]:
add.invoke({"a": 3, "b":5})

8

LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아보자

In [5]:
query = '3 곱하기 5는?'

llm 사용

In [6]:
small_llm.invoke(query)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed70d1b7-74e5-4255-b2e8-dc3de3b1f01e-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

도구 사용

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달

In [7]:
llm_with_tools = small_llm.bind_tools([add, multiply])

- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [8]:
result = llm_with_tools.invoke(query)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n2JIWPlyh4O3Qkplu0bUuhmx', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c88d99c-7047-4dc8-86a1-4dcfee4a8e7c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_n2JIWPlyh4O3Qkplu0bUuhmx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- AIMessage content는 비어있고, additional_kwargs에 tool_calls가 추가되었다.

(중요)여기서 `tool_calls`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 된다.

In [9]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_n2JIWPlyh4O3Qkplu0bUuhmx',
  'type': 'tool_call'}]

- 현재는 LLM에게 도구를 연결해준 상태이다.  
- 이제 LLM이 도구를 invoke 할수 있게 해야한다????

LLM이 도구를 사용해서 정상적으로 답변하도록 하려면 메시지 리스트 포맷으로 입력하여 invoke해야한다.  
- 예: [(Human: dfsdf), (AI: sfsdf), (Tool:fsdfsd)]

In [10]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 


- AnyMessage의 종류에는 다음이 있다.
    - SystemMessage, HumanMessage, AIMessage, 그리고 ToolMessage

- `tool_calls` 속성은 도구를 호출하는 메시지를 포함함
- `tool_calls`를 가진 `AIMessage`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 됨

이제 다시 invoke 해보자.

In [11]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7uNTSwrl0fcbwxbFLkWWln4o', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-24c87976-bb40-43bc-9723-65d6cc53913e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_7uNTSwrl0fcbwxbFLkWWln4o', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_7uNTSwrl0fcbwxbFLkWWln4o',
  'type': 'tool_call'}]

- 이전과 동일하다.   
- 그런데 이것은 AI 메시지이므로 그 결과를 다시 메시지 리스트에 추가한다.

In [13]:
message_list.append(ai_message)

(중요)`AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있음

In [14]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

In [15]:
tool_message

ToolMessage(content='15', name='multiply', tool_call_id='call_7uNTSwrl0fcbwxbFLkWWln4o')

- ToolMessage가 생성됨

하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달한다.

In [16]:
message_list.append(tool_message)

In [17]:
llm_with_tools.invoke(message_list)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-28704119-c9c7-4f5e-be38-b203ca00411b-0', usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- `message_list`의 순서를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [18]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7uNTSwrl0fcbwxbFLkWWln4o', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-24c87976-bb40-43bc-9723-65d6cc53913e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_7uNTSwrl0fcbwxbFLkWWln4o', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0